In [1]:
#CSE 494 Lab 1: Unsupervised Machine Learning Demo (Using K-means Clustering)
#Author: Anay Gupta

In [2]:
#importing necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler 

In [3]:
#reads in csv file and returns dataframe containing feature columns and class label columns
#--rows are samples
#--columns are features for k-means (last column indicates the class label)
def preProcessData(file):
    df = pd.read_csv(file)
    
    #rename Unnamed to University
    df.rename(columns={'Unnamed: 0': 'University'}, inplace = True)
    df['Grad.Rate.Bins'] = pd.cut(df['Grad.Rate'], 6, labels = list(range(6)))
    
    df.Private[df.Private == 'Yes'] = 1
    df.Private[df.Private == 'No'] = 0
    
    #store schools in a dictionary and drop that column
    schools_dict = df['University'].to_dict()
    print schools_dict[23]
    df = df.drop('University',axis=1)
    
    list_target = ['Grad.Rate.Bins']
    list_to_correlate = list(df.columns.values)[:1]
    
    df_corr = df.apply(lambda x: pd.factorize(x)[0]).corr(method='pearson', min_periods = 1)
    corr = pd.DataFrame()
    
    for a in list_target:
        for b in list_to_correlate:
            corr.loc[a,b] = df_corr.loc[a,b]
            
#     print(corr)
    
    columns_to_select = ['Top10perc','Top25perc','perc.alumni','Enroll','Grad.Rate.Bins']
    
    df_sel = df[columns_to_select]
    
    print df_sel.head
    
    return df_sel, schools_dict.

In [4]:
#calculates Euclidean distance
def _distance(a,b):
    return np.sqrt(((a-b)**2).sum())

In [5]:
#first, find the sum of the distances from each sample to its respective centroid for each cluster
#then, return the minimum value
def _nearest(clusters,x):
    return np.argmin([_distance(x,c) for c in clusters])

In [6]:
#running Kmeans
def runKmeans(X, n_clust, init_method, max_iter_num):
    kmeans = KMeans(n_clusters = n_clust, init = init_method, verbose = 0, tol = 1e-3, max_iter = max_iter_num, n_init = 10)
    kmeans.fit(X)
    
    return kmeans

In [7]:
def findIndexOfNearestCluster(model, df, schools_dict, school_name):
    for idx, s in schools_dict.items():
        if s == school_name:
            feat_school = df.iloc[idx].values

    clus_cent = model.cluster_centers_

    print(_nearest(clus_cent, feat_school.T))

In [8]:
def evaluateKMeans(df, schools_dict):

    df = df.drop('Grad.Rate.Bins', axis=1)  # Remove the column corresponding to the class label
    
    X = df.values
    print X
    model = runKmeans(X, 6, 'random', 300)

    print("Silhouette scores evaluation of K means on unscaled data ")
    print(silhouette_score(X, model.labels_))
    
    print 'Model Inertia:'
    print (model.inertia_)
    
    plt.figure(figsize=(10, 7))  
    plt.scatter(X[:,1], X[:,2], c=model.labels_, cmap='rainbow') # Inertia -  the sum of squared distances of samples to the nearest cluster centre
    plt.show()

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    print X_scaled
    
    model = runKmeans(X_scaled, 3, 'random', 300)
    
    print("Silhouette scores evaluation of K means on scaled data ")
    print(silhouette_score(X_scaled, model.labels_))
    
    print 'Model Inertia:'
    print (model.inertia_)
    temp = findIndexOfNearestCluster(model, df, schools_dict, 'Arizona State University Main Campus')
    print temp

#     return model
    


In [3]:
if __name__ == "__main__":
    file = '/Volumes/GoogleDrive/My Drive/Spring 19/CSE 494 Labs/Lab 1/College.csv'


    df, schools_dict = preProcessData(file)

    evaluateKMeans(df, schools_dict)
    

/Users/anaygupta/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/anaygupta/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Arizona State University Main campus
<bound method DataFrame.head of      Top10perc  Top25perc  perc.alumni  Enroll Grad.Rate.Bins
0           23         52           12     721              2
1           16         29           16     512              2
2           22         50           30     336              2
3           60         89           37     137              2
4           16         44            2      55              0
5           38         62           11     158              2
6           17         45           26     103              2
7           37         68           37     489              3
8           30         63           23     227              3
9           21         44           15     172              2
10          37         75           31     472              3
11          44         77           41     484              3
12          38         64           21     290              3
13          44         73           32     385              3
1

NameError: name 'evaluateKMeans' is not defined